In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Sep 23 17:46:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
! pip install ultralytics

In [5]:
from ultralytics import YOLO

In [6]:
!ls /content/drive/MyDrive/Viridien/

seep_detection	task_1	task_2


In [6]:
import torch

In [8]:
! pwd

/content


# Training

## Task 1 (seep or non-seep)

In [13]:
model = YOLO('yolov8l-seg.pt')
model.model.nc = 1

In [14]:
import yaml

yaml_content = {
    'path': '/content/drive/MyDrive/Viridien/task_1/dataset',
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': 1,
    'names': {
        0: 'seep',

    },
    # Data augmentation parameters
    'hsv_h': 0.015,  # Hue augmentation
    'hsv_s': 0.7,    # Saturation augmentation
    'hsv_v': 0.4,    # Value (brightness) augmentation
    'degrees': 90,    # Image rotation (+/- deg)
    'translate': 0.1,  # Image translation (+/- fraction)
    'scale': 0.5,    # Image scale (+/- gain)
    'shear': 0.0,    # Image shear (+/- deg)
    'perspective': 0.0,  # Image perspective (+/- fraction), range 0-0.001
    'flipud': 0.5,   # Image flip up-down (probability)
    'fliplr': 0.5,   # Image flip left-right (probability)
    'mosaic': 1.0,   # Image mosaic (probability)
    'mixup': 0.5,    # Image mixup (probability)
    'copy_paste': 0.5,  # Segment copy-paste (probability)


    'patience': 20

}


with open('/content/drive/MyDrive/Viridien/task_1/data.yaml', 'w') as file:
    yaml.dump(yaml_content, file, default_flow_style=False)

In [15]:
torch.cuda.is_available(), torch.cuda.device_count()

(True, 1)

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [17]:
model.train(data='/content/drive/MyDrive/Viridien/task_1/data.yaml',
            epochs=300, lr0=0.0001, lrf=0.01, imgsz=256, device='0',
            project='/content/drive/MyDrive/Viridien/task_1/runs',
            name='experiment_1_3',
            pretrained=True)

Ultralytics YOLOv8.2.99 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=segment, mode=train, model=yolov8l-seg.pt, data=/content/drive/MyDrive/Viridien/task_1/data.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/Viridien/task_1/runs, name=experiment_1_3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sav

train: Scanning /content/drive/MyDrive/Viridien/task_1/dataset/labels/train.cache... 1160 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1160/1160 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Viridien/task_1/dataset/labels/val.cache... 238 images, 0 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/Viridien/task_1/runs/experiment_1_3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 106 weight(decay=0.0), 117 weight(decay=0.0005), 116 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/Viridien/task_1/runs/experiment_1_3
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        238        741      0.815      0.546       0.63      0.418      0.699      0.487       0.54      0.193



300 epochs completed in 1.307 hours.
Optimizer stripped from /content/drive/MyDrive/Viridien/task_1/runs/experiment_1_3/weights/last.pt, 92.3MB
Optimizer stripped from /content/drive/MyDrive/Viridien/task_1/runs/experiment_1_3/weights/best.pt, 92.3MB

Validating /content/drive/MyDrive/Viridien/task_1/runs/experiment_1_3/weights/best.pt...
Ultralytics YOLOv8.2.99 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,912,659 parameters, 0 gradients, 220.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.23it/s]


                   all        238        741      0.817      0.547      0.632       0.42      0.759      0.462       0.54      0.193
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /content/drive/MyDrive/Viridien/task_1/runs/experiment_1_3


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c8836d255d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

## Task 2 (seven classes)

In [7]:
model = YOLO('/content/drive/MyDrive/Viridien/task_1/runs/experiment_1_3/weights/best.pt')
model.model.nc = 7
model.model.model[-1].nc = 7

In [8]:
import yaml

yaml_content = {
    'path': '/content/drive/MyDrive/Viridien/task_2/dataset',
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': 7,
    'names': {
        0: 'class1',
        1: 'class2',
        2: 'class3',
        3: 'class4',
        4: 'class5',
        5: 'class6',
        6: 'class7'
    },
    # Data augmentation parameters
    'hsv_h': 0.015,  # Hue augmentation
    'hsv_s': 0.7,    # Saturation augmentation
    'hsv_v': 0.4,    # Value (brightness) augmentation
    'degrees': 90,    # Image rotation (+/- deg)
    'translate': 0.1,  # Image translation (+/- fraction)
    'scale': 0.5,    # Image scale (+/- gain)
    'shear': 0.0,    # Image shear (+/- deg)
    'perspective': 0.0,  # Image perspective (+/- fraction), range 0-0.001
    'flipud': 0.5,   # Image flip up-down (probability)
    'fliplr': 0.5,   # Image flip left-right (probability)
    'mosaic': 1.0,   # Image mosaic (probability)
    'mixup': 0.5,    # Image mixup (probability)
    'copy_paste': 0.5,  # Segment copy-paste (probability)


    'patience': 20
}

with open('/content/drive/MyDrive/Viridien/task_2/data.yaml', 'w') as file:
    yaml.dump(yaml_content, file, default_flow_style=False)

In [9]:
torch.cuda.is_available(), torch.cuda.device_count()

(True, 1)

In [10]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [12]:
model.train(data='/content/drive/MyDrive/Viridien/task_2/data.yaml',
            epochs=200, lr0=0.0001, lrf=0.01, imgsz=256, device='0',
            project='/content/drive/MyDrive/Viridien/task_2/runs',
            name='experiment_2_3',
            pretrained=True)

Ultralytics YOLOv8.2.99 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=segment, mode=train, model=/content/drive/MyDrive/Viridien/task_1/runs/experiment_1_3/weights/best.pt, data=/content/drive/MyDrive/Viridien/task_2/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/Viridien/task_2/runs, name=experiment_2_3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina

train: Scanning /content/drive/MyDrive/Viridien/task_2/dataset/labels/train.cache... 1160 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1160/1160 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Viridien/task_2/dataset/labels/val.cache... 238 images, 0 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/Viridien/task_2/runs/experiment_2_3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 106 weight(decay=0.0), 117 weight(decay=0.0005), 116 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/Viridien/task_2/runs/experiment_2_3
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        238        741      0.612      0.285       0.36      0.241      0.532      0.246      0.278      0.108



200 epochs completed in 0.847 hours.
Optimizer stripped from /content/drive/MyDrive/Viridien/task_2/runs/experiment_2_3/weights/last.pt, 92.3MB
Optimizer stripped from /content/drive/MyDrive/Viridien/task_2/runs/experiment_2_3/weights/best.pt, 92.3MB

Validating /content/drive/MyDrive/Viridien/task_2/runs/experiment_2_3/weights/best.pt...
Ultralytics YOLOv8.2.99 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,917,285 parameters, 0 gradients, 220.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.25it/s]


                   all        238        741      0.601      0.286      0.361      0.241      0.528      0.246      0.279       0.11
                class1        131        335      0.717      0.582      0.644      0.501      0.686      0.553      0.577      0.232
                class2         61         93       0.56      0.398       0.45      0.328      0.495      0.349      0.339      0.121
                class3         27         37      0.762      0.378      0.455      0.276      0.493      0.243      0.224     0.0853
                class4         13         17      0.291     0.0588     0.0861     0.0645      0.299     0.0588     0.0847     0.0342
                class5         26         82      0.653      0.232      0.282      0.171      0.557      0.195      0.249      0.102
                class6         64        146      0.735      0.323      0.455      0.285       0.66      0.288       0.37      0.146
                class7         19         31      0.492     0.0323   

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7abfc19089d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04

In [13]:
from google.colab import runtime
runtime.unassign()

# Evaluation

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Sep 24 01:11:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.1/874.1 kB 27.2 MB/s eta 0:00:00


In [7]:
import torch
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
model = YOLO('/content/drive/MyDrive/Viridien/task_1/runs/experiment_1_3/weights/best.pt')

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Task 1

In [10]:
model.val(data='/content/drive/MyDrive/Viridien/task_1/data.yaml',
            project='/content/drive/MyDrive/Viridien/task_1/test',
            name='test_1_3')

YOLOv8l-seg summary (fused): 295 layers, 45,912,659 parameters, 0 gradients, 220.1 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 23.7MB/s]
val: Scanning /content/drive/MyDrive/Viridien/task_1/dataset/labels/val.cache... 238 images, 0 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:48<00:00,  3.22s/it]


                   all        238        741      0.811      0.549      0.634      0.419      0.682      0.482      0.526      0.188
Speed: 0.1ms preprocess, 9.7ms inference, 0.0ms loss, 4.9ms postprocess per image
Results saved to /content/drive/MyDrive/Viridien/task_1/test/test_1_3


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc1ea1c8430>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

## Task 2

In [11]:
model = YOLO('/content/drive/MyDrive/Viridien/task_2/runs/experiment_2_3/weights/best.pt')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [12]:
model.val(data='/content/drive/MyDrive/Viridien/task_2/data.yaml',
            project='/content/drive/MyDrive/Viridien/task_2/test',
            name='test_2_3')

YOLOv8l-seg summary (fused): 295 layers, 45,917,285 parameters, 0 gradients, 220.2 GFLOPs


val: Scanning /content/drive/MyDrive/Viridien/task_2/dataset/labels/val.cache... 238 images, 0 backgrounds, 0 corrupt: 100%|██████████| 238/238 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:25<00:00,  1.68s/it]


                   all        238        741      0.604      0.286       0.36      0.241      0.527      0.244      0.273      0.108
                class1        131        335      0.725      0.582      0.645      0.502      0.683      0.547      0.571      0.228
                class2         61         93       0.56      0.397       0.45      0.332      0.495      0.348      0.334      0.119
                class3         27         37      0.763      0.378      0.455      0.272      0.494      0.243      0.224     0.0855
                class4         13         17      0.294     0.0588     0.0861     0.0642      0.303     0.0588     0.0847     0.0339
                class5         26         82      0.655      0.232      0.283      0.175      0.559      0.195      0.243      0.102
                class6         64        146      0.735      0.322      0.448      0.284      0.647      0.281      0.345      0.139
                class7         19         31      0.494     0.0323   

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fc142970b50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04